<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/Copy_of_Generating_Afrobeats_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing necessary libraries
!pip install sentence-transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=1ba72bd47829a80ca994df41c1b27b9baf048d22f2557993f6cddf8ebd9c7ea6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#songs csv 
!wget https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/spotify_transformed.csv

--2023-01-23 21:18:57--  https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/spotify_transformed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167653 (164K) [text/plain]
Saving to: ‘spotify_transformed.csv’

spotify_transformed 100%[===================>] 163.72K  --.-KB/s    in 0.02s   

2023-01-23 21:18:57 (9.84 MB/s) - ‘spotify_transformed.csv’ saved [167653/167653]



In [ ]:
#setting up wandb
!wandb login
import wandb

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
config = {
    'Name' : 'Embeddings for afrobeats lyrics',
    'Dataset' : 'spotify api lyrics',
    'Model' : 'msmarco-miniLM-pretrained',
    'notes':'No text cleaning was performed yet.creating a baseline performance'
}
wandb.init(project='Playlist Generator',entity='damilojohn',config=config)

In [3]:
from sentence_transformers import SentenceTransformer,util
from tqdm.auto import tqdm
import pandas as pd 
import transformers
import pickle 
from huggingface_hub import hf_hub_download
import re

In [7]:
#loading song lyrics into a dataset object 
data_path = '/content/spotify_transformed.csv'

In [5]:
embedder = SentenceTransformer('msmarco-distilbert-base-v4')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [8]:
songs= pd.read_csv(data_path)

In [17]:
songs.head(10)

,id,album,artist,song_id,song_name,image,url,for_lyrics
0,3tVQdUvClmAT7URs9V3rsp,"More Love, Less Ego",Wizkid,1DA2ADZs6O28y2rmdmpekw,2 Sugar (feat. Ayra Starr),https://i.scdn.co/image/ab67616d0000b273e944c5...,https://open.spotify.com/track/1DA2ADZs6O28y2r...,[Verse 1: Ayra Starr]\nGot a bag full of feeli...
1,3tVQdUvClmAT7URs9V3rsp,Made In Lagos,Wizkid,5FG7Tl93LdH117jEKYl3Cm,Essence (feat. Tems),https://i.scdn.co/image/ab67616d00001e02e944c5...,https://open.spotify.com/track/5FG7Tl93LdH117j...,"[Intro: Wizkid]\nOh-na-na, omo je ka shi ge ge..."
2,3tVQdUvClmAT7URs9V3rsp,Made In Lagos: Deluxe Edition,Wizkid,6jdTkoEaer7XNGSblczoSu,Mood (feat. BNXN),https://i.scdn.co/image/ab67616d00001e0290e89e...,https://open.spotify.com/track/6jdTkoEaer7XNGS...,[Intro: Buju]\nMmm-mmm\nWey come cooli my body...
3,3tVQdUvClmAT7URs9V3rsp,Made In Lagos,Wizkid,51mLQ3w7yR7vjdSTFLWaY5,Ginger (feat. Burna Boy),https://i.scdn.co/image/ab67616d0000485190e89e...,https://open.spotify.com/track/51mLQ3w7yR7vjdS...,[Chorus: Burna Boy & Wizkid]\nIf you want make...
4,3tVQdUvClmAT7URs9V3rsp,"More Love, Less Ego",Wizkid,2HYcI74UpbMMxLZ1m7Xnfn,Bad To Me,https://i.scdn.co/image/ab67616d0000b273e944c5...,https://open.spotify.com/track/2HYcI74UpbMMxLZ...,"[Verse 1]\nHiye, burst it my way\nThis kind of..."
5,3tVQdUvClmAT7URs9V3rsp,We're All Alone In This Together,Wizkid,5gOtq0sKU24ZF8hnXlSp1V,System (feat. WizKid),https://i.scdn.co/image/ab67616d00001e02e944c5...,https://open.spotify.com/track/5gOtq0sKU24ZF8h...,[]
6,3tVQdUvClmAT7URs9V3rsp,Made In Lagos: Deluxe Edition,Wizkid,1MZtr7IH5qtjIkqrXj8WOJ,Essence (feat. Justin Bieber & Tems),https://i.scdn.co/image/ab67616d00004851e944c5...,https://open.spotify.com/track/1MZtr7IH5qtjIkq...,"[Intro: Wizkid]\nOh-na-na, omo je ka shi ge ge..."
7,3tVQdUvClmAT7URs9V3rsp,Joro,Wizkid,3Yt1ovsh3v3VEzRuhI1TL5,Joro,https://i.scdn.co/image/ab67616d0000b273a19e09...,https://open.spotify.com/track/3Yt1ovsh3v3VEzR...,"[Chorus]\nJoro, joro, joro, joro, joro, joro, ..."
8,0Y3agQaa6g2r0YmHPOO9rh,Hayya Hayya (Better Together) [Music from FIFA...,Davido,7vAJiDFrFtMYVOrZbOw7jj,Hayya Hayya (Better Together) - Music from the...,https://i.scdn.co/image/ab67616d00001e02a19e09...,https://open.spotify.com/track/7vAJiDFrFtMYVOr...,[]
9,0Y3agQaa6g2r0YmHPOO9rh,Electricity,Davido,6McBvfxgkVfREmQPwTKBUn,Electricity,https://i.scdn.co/image/ab67616d00004851a19e09...,https://open.spotify.com/track/6McBvfxgkVfREmQ...,[]


In [10]:
def clean_verses(lyrics_uncleaned):
  ''' takes a list of verses as input and removes useless characters such as /n and stuff '''
  lyrics_cleaned = lyrics_uncleaned.split('\n')
  lyrics_cleaned = [i for i in lyrics_cleaned if i]
  for i,lyrics in enumerate(lyrics_cleaned) :
    if not lyrics[0].isalpha():
      del lyrics_cleaned[i]
  return lyrics_cleaned 


In [52]:
embeddings = embedder.encode(new_verses,)
assert len(new_verses) == len(embeddings) , 'embeddings and lyrics are mismatched '

In [53]:
#trying out semantic search 
query = ['create a playlist about a sunny afternoon on the beach']
query_embedding = embedder.encode(query)
hits = util.semantic_search(query_embedding,embeddings,top_k = 6)
hits

[[{'corpus_id': 1423, 'score': 0.5428035259246826},
  {'corpus_id': 1422, 'score': 0.4128659963607788},
  {'corpus_id': 1455, 'score': 0.37966078519821167},
  {'corpus_id': 1421, 'score': 0.3533474802970886},
  {'corpus_id': 3707, 'score': 0.32062721252441406},
  {'corpus_id': 3665, 'score': 0.32062721252441406}]]

In [56]:
new_verses[1423]

'Kissing in the early morning beach'

In [54]:
new_dict[new_verses[1423]]

'Time N Affection'

In [23]:
sample = songs

In [26]:
sample['for_lyrics'] = sample['for_lyrics'].apply(clean_verses,)

In [ ]:
sample.head(5)

In [ ]:
#trying out regex later 
pattern = re.sub('\[')


In [28]:
dictt=dict(zip(sample.song_name,sample.for_lyrics))

In [ ]:
verses = []
for i in sample['for_lyrics'].values:
  verses.append(i)
verses = [i for i in verses if i] 
new_verses = [] 
for i in verses :
  for j in i :
    new_verses.append(j)
new_verses

In [ ]:
verses

In [ ]:
dictt['2 Sugar (feat. Ayra Starr)']

In [38]:
new_dict={}
for i,j in dictt.items():
    for k in j:
        new_dict[k]=i

In [ ]:
import pickle
path = '/content/OG_embeddings.pkl'
with open(path,'wb') as pk:
  pickle.dump(embeddings,pk)

In [ ]:
from huggingface_hub import login 
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
#borrowing this space to learn regular expressions 

In [ ]:
import re

In [ ]:
text = 'Call me at 415-555-1011 tomorrow. 415-555-9999 is my office.'

In [ ]:
reg = re.compile(r'\d{3}-\d{3}-\d{4}')
finds = reg.search(text)
finds.group()

'415-555-1011'

In [ ]:
heroregex = re.compile('batman|Tina Fey')
match1 = heroregex.search(' the batman had Tina Fey in it ')
match1.group()

'batman'